In [1]:
import sys
sys.path.extend(['..', '../', '../..', '../../' ])
%load_ext autoreload
%autoreload 2

# LeNet5 - Malaria Cells

In [2]:
import paths
import keras
from src.parameters import metrics
from src.datasets.dataset_generators import DatasetGenerators
from src.keras_callback.time_history import TimeHistory
from src.export.csv_export import export_learning_history_to_csv, export_learning_validation_result_to_csv
from src.export.model_export import clear_exported_models
from src.keras_model.lenet5_network import LeNet5Network

Using TensorFlow backend.


In [3]:
network_name = 'lenet5'
fit_epochs = 50
dataset = 'malaria-cells'
dataset_percentage = '5'

In [4]:
network = LeNet5Network()

In [5]:
dataset_generators = DatasetGenerators(
    name=f'{dataset}_{dataset_percentage}',
    dataset_dir=paths.data_location,
    image_size=network.image_size,
    batch_size=network.batch_size,
    grayscale=network.grayscale
)

Loading train set for malaria-cells_5...
Found 966 images belonging to 2 classes.
Loading test set for malaria-cells_5...
Found 276 images belonging to 2 classes.
Loading validation set for malaria-cells_5...
Found 138 images belonging to 2 classes.


In [6]:
model = network.get_compiled_model(
    classes=dataset_generators.test.num_classes
)

In [7]:
checkpoint_path = f'{paths.models_location}{network_name}/{dataset}/'
best_model_path = f'{checkpoint_path}best.h5'
clear_exported_models(checkpoint_path)

checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=f'{checkpoint_path}{{epoch:02d}}-{{val_accuracy}}.h5',
    save_best_only=False,
    save_weights_only=False
)

best_callback = keras.callbacks.ModelCheckpoint(
    filepath=best_model_path,
    save_best_only=True,
    verbose=1,
    save_weights_only=False
)

time_callback = TimeHistory()

model_callbacks = [checkpoint_callback, best_callback, time_callback]

In [8]:
fit_result = model.fit_generator(
    generator=dataset_generators.train,
    epochs=fit_epochs,
    validation_data=dataset_generators.test,
    callbacks = model_callbacks
)

Epoch 1/50
31/31 [==============================] - 2s 67ms/step - loss: 0.7981 - accuracy: 0.5745 - precision: 0.5590 - recall: 0.5766 - val_loss: 0.5603 - val_accuracy: 0.6449 - val_precision: 0.6175 - val_recall: 0.7641

Epoch 00001: val_loss improved from inf to 0.56035, saving model to /Users/igor/Python/CNN-Architecture-Comparison/models/lenet5/malaria-cells/best.h5
Epoch 2/50
31/31 [==============================] - 1s 46ms/step - loss: 0.5751 - accuracy: 0.7081 - precision: 0.7207 - recall: 0.7518 - val_loss: 0.7214 - val_accuracy: 0.7174 - val_precision: 0.6954 - val_recall: 0.7866

Epoch 00002: val_loss did not improve from 0.56035
Epoch 3/50
31/31 [==============================] - 1s 46ms/step - loss: 0.4665 - accuracy: 0.7919 - precision: 0.7702 - recall: 0.8475 - val_loss: 0.6693 - val_accuracy: 0.7645 - val_precision: 0.8457 - val_recall: 0.6332

Epoch 00003: val_loss did not improve from 0.56035
Epoch 4/50
31/31 [==============================] - 2s 57ms/step - loss: 0.

In [9]:
best_model = keras.models.load_model(best_model_path,
    custom_objects= {
        'precision': metrics.precision,
        'recall': metrics.recall
    }
)
validation_result = best_model.evaluate_generator(
    generator=dataset_generators.validate
)

## Results

### Epoch CSV

In [10]:
csv_file_path = f'{paths.results_location}{network_name}/{dataset}-history.csv'
export_learning_history_to_csv(csv_file_path, fit_result, time_callback.times)

epoch, time, val_loss, val_accuracy, val_precision, val_recall
0,0,0,0,0,0
1,2.190312147140503,0.560345470905304,0.6449275612831116,0.6175258159637451,0.7641382217407227
2,1.4623489379882812,0.7213678956031799,0.717391312122345,0.6954374313354492,0.7866097092628479
3,1.4621691703796387,0.6692983508110046,0.7644927501678467,0.8457061648368835,0.6332299709320068
4,1.7867991924285889,1.135623574256897,0.70652174949646,0.8845679759979248,0.46676328778266907
5,2.0211880207061768,0.8061447143554688,0.7463768124580383,0.7437801957130432,0.7508230209350586
6,1.9776270389556885,0.33477431535720825,0.739130437374115,0.7676001191139221,0.7232851982116699
7,2.0036628246307373,0.4050665497779846,0.7427536249160767,0.7433984279632568,0.7369751930236816
8,1.9339630603790283,0.37809988856315613,0.760869562625885,0.7496327757835388,0.7629311084747314
9,2.2632691860198975,0.5157225728034973,0.7681159377098083,0.7820661664009094,0.7503905296325684
10,2.0489141941070557,0.5445934534072876,0.77536231279373

### Validation result

In [11]:
csv_file_path = f'{paths.results_location}{network_name}/{dataset}-result.csv'
export_learning_validation_result_to_csv(csv_file_path, validation_result, time_callback.times)

avg_epoch_time, time, val_loss, val_accuracy, val_precision, val_recall
1.702968339920044,85.1484169960022,2.9245383739471436,0.8478260636329651,0.8294677734375,0.8196031451225281
